In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchsummary import summary
from torchvision import transforms,datasets
from torch.utils.data import TensorDataset, DataLoader, Dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#====================================================================
# calling data 
trans = transforms.Compose([transforms.Resize((224,224)),
#                             transforms.RandomHorizontalFlip(p=0.5), #data augmentation
#                             transforms.RandomVerticalFlip(p=0.5),   #data augmentation
                            transforms.ToTensor(),
                            transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset = torchvision.datasets.ImageFolder(root='/home/gus/PyTorch/256_ObjectCategories',
                                           transform=trans)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=32,
                                          shuffle=True,
                                          num_workers=2)
# testset = torchvision.datasets.Caltech101(root='/home/gus/PyTorch',
#                                           target_type='category',
#                                           transform=trans)
# testloader = torch.utils.data.DataLoader(testset,
#                                          batch_size=64,
#                                          shuffle=False,
#                                          num_workers=0)
#====================================================================
# i = 0
# a = 0
# b = 0
# for X, Y in enumerate(trainloader):
# #     print(X) #batch number
# #     print(Y[0].shape) #Y[0] -> data
# #     print(Y[1])       #Y[1] -> category
#     a = Y[0][0]
#     b = Y[1][0]
#     break
# def imshow(img):
#     img = img / 2 + 0.5     # unnormalize
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()
    

# imshow(torchvision.utils.make_grid(a))
#====================================================================
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16,self).__init__()
        
        self.layer1 = nn.Sequential(
            # input shape = (?, 3, 224, 224)
            # Conv -> (?, 64, 224, 224)
            torch.nn.Conv2d(3,64,3,stride=1,padding=1),
            torch.nn.ReLU(),
            # Conv -> (?, 64, 224, 224)
            # Pool -> (?, 64, 112, 112)
            torch.nn.Conv2d(64,64,3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,stride=2)
        )
        self.layer2 = nn.Sequential(
            # input shape = (?, 64, 112, 112)
            # Conv -> (?, 128, 112, 112)
            torch.nn.Conv2d(64,128,3,stride=1,padding=1),
            torch.nn.ReLU(),
            # Conv -> (?, 128, 112, 112)
            # Pool -> (?, 128, 56, 56)
            torch.nn.Conv2d(128,128,3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,stride=2)
        )
        self.layer3 = nn.Sequential(
            # input shape = (?, 128, 56, 56)
            # Conv -> (?, 256, 56, 56)           
            torch.nn.Conv2d(128,256,3,stride=1,padding=1),
            torch.nn.ReLU(),
            # input shape = (?, 256, 56, 56)
            # Conv -> (?, 256, 56, 56)           
            torch.nn.Conv2d(256,256,3,stride=1,padding=1),
            torch.nn.ReLU(),
            # input shape = (?, 256, 56, 56)
            # Conv -> (?, 256, 56, 56)
            # Pool -> (?, 256, 28, 28)
            torch.nn.Conv2d(256,256,3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,stride=2)
        )
        self.layer4 = nn.Sequential(
            # input shape = (?, 256, 28, 28)
            # Conv -> (?, 512, 28, 28)           
            torch.nn.Conv2d(256,512,3,stride=1,padding=1),
            torch.nn.ReLU(),
            # input shape = (?, 512, 28, 28)
            # Conv -> (?, 512, 28, 28)           
            torch.nn.Conv2d(512,512,3,stride=1,padding=1),
            torch.nn.ReLU(),
            # input shape = (?, 512, 28, 28)
            # Conv -> (?, 512, 28, 28)
            # Pool -> (?, 512, 14, 14)
            torch.nn.Conv2d(512,512,3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,stride=2)
        )
        self.layer5 = nn.Sequential(
            # input shape = (?, 512, 14, 14)
            # Conv -> (?, 512, 14, 14)           
            torch.nn.Conv2d(512,512,3,stride=1,padding=1),
            torch.nn.ReLU(),
            # input shape = (?, 512, 14, 14)
            # Conv -> (?, 512, 14, 14)           
            torch.nn.Conv2d(512,512,3,stride=1,padding=1),
            torch.nn.ReLU(),
            # input shape = (?, 512, 14, 14)
            # Conv -> (?, 512, 14, 14)
            # Pool -> (?, 512, 7, 7)
            torch.nn.Conv2d(512,512,3,stride=1,padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2,stride=2)
        )
        self.layer6 = nn.Sequential(
            torch.nn.Dropout(p=0.5),                          #Implementing Dropout
            torch.nn.Linear(7*7*512,4096,bias=True),
            torch.nn.ReLU()
        )
        self.layer7 = nn.Sequential(
            torch.nn.Dropout(p=0.5),                          #Implementing Dropout
            torch.nn.Linear(4096,4096,bias=True),
            torch.nn.ReLU()
        )
        self.layer8 = nn.Sequential(
            torch.nn.Linear(4096,256,bias=True)
        )
        
        
    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=self.layer3(out)
        out=self.layer4(out)
        out=self.layer5(out)
        out=out.view(out.size(0),-1)
        out=self.layer6(out)
        out=self.layer7(out)
        out=self.layer8(out)
        return out
    
    def _initialize_weights(self):
        # .modules로 sequential에 있는 layer를 하나하나 불러올 수 있습니다.
        for m in self.modules():
            print(m)
            # isinstance 함수로 m이 nn.Conv2d인지 확인합니다.
            if isinstance(m, nn.Conv2d):
                # He 가중치 초기화(relu에 최적화되어있는 가중치 초기화 기법)
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not Node:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
model = VGG16().to(device)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [12]:
from torchsummary import summary
summary(model, input_size=(3, 224, 224))

RuntimeError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 1.95 GiB total capacity; 1.04 GiB already allocated; 4.25 MiB free; 1.07 GiB reserved in total by PyTorch)